What we do here is load in the per gene file generated by the command **python compare_observed_and_expected_mut_burden.py casesFile mode** and reason on it

In [1]:
import pandas as pd
import os
import sys

In [2]:
sys.path.append('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util

In [3]:
pathPrefix = '/Users/friedman/Desktop/mnt'

In [4]:
reload(mutation_modeling_util)

<module 'mutation_modeling_util' from '/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils/mutation_modeling_util.pyc'>

In [5]:
df = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/simulatedMutationSummary.tsv')

In [6]:
oncogenicityDict = mutation_modeling_util.calculate_quadnuc_based_oncogenic_susceptibility_dict(df)


In [7]:
oncogenicityDictByGene = mutation_modeling_util.calculate_quadnuc_and_gene_based_oncogenic_susceptibility_dict(df)

considering the  25 th gene
considering the  50 th gene
considering the  75 th gene
considering the  100 th gene
considering the  125 th gene
considering the  150 th gene
considering the  175 th gene
considering the  200 th gene
considering the  225 th gene
considering the  250 th gene
considering the  275 th gene
considering the  300 th gene
considering the  325 th gene
considering the  350 th gene
considering the  375 th gene
considering the  400 th gene
considering the  425 th gene
considering the  450 th gene


In [27]:
allImpactMuts = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,36,39,85,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Now enumerate observed vs expected data for genes based on their type and cancer type**

we run the following on the cluster:
python compare_observed_and_expected_mut_burden.py /ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/idFiles/endometrialHypermutantIds.txt perGene

python compare_observed_and_expected_mut_burden.py /ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/idFiles/colorectalHypermutantIds.txt perGene

python compare_observed_and_expected_mut_burden.py /ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/idFiles/endometrialHypermutantIds.txt perGene outfile


In [71]:
endometrialDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/observedVsExpected/endometrialGenesObservedVsExpected.tsv')
colorectalDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/observedVsExpected/colorectalGenesObservedVsExpected.tsv')
gliomaDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/observedVsExpected/gliomaGenesObservedVsExpected.tsv')

In [72]:
cancerTypeImplicationDictBig = maf_analysis_utils.create_dictionary_mapping_genes_to_cancer_types_with_implication(allImpactMuts, pathPrefix, cancerTypes=set(['Glioma', 'Colorectal Cancer', 'Endometrial Cancer']), t=.1)
cancerTypeImplicationDictSmall = maf_analysis_utils.create_dictionary_mapping_genes_to_cancer_types_with_implication(allImpactMuts, pathPrefix, cancerTypes=set(['Glioma', 'Colorectal Cancer', 'Endometrial Cancer']), t=.025) 
for key, value in cancerTypeImplicationDictBig.items():
    curCTypeGenes = cancerTypeImplicationDictSmall[key]
    cancerTypeImplicationDictSmall[key] = curCTypeGenes - value


Endometrial Cancer
Glioma
Colorectal Cancer
Endometrial Cancer
Glioma
Colorectal Cancer


In [73]:
endometrialDf['cancer_type'] = 'Endometrial Cancer'
colorectalDf['cancer_type'] = 'Colorectal Cancer'
gliomaDf['cancer_type'] = 'Glioma'
combinedDf = pd.concat([endometrialDf, colorectalDf, gliomaDf])

In [74]:
combinedDf['geneMutType'] = combinedDf.apply(lambda row:
                                            'strongly_recurrent' if row['gene'] in cancerTypeImplicationDictBig[row['cancer_type']]
                                             else 'weakly_recurrent' if row['gene'] in cancerTypeImplicationDictSmall[row['cancer_type']]
                                            else 'not_recurrent', axis=1
                                            )

In [75]:
combinedDf['nCases'] = combinedDf['cancer_type'].apply(lambda x: 53 if x == 'Glioma'
                                                       else 261 if x == 'Colorectal Cancer' 
                                                       else 237)

In [ ]:
combinedDf['geneObsExpectedRatio'] = combinedDf.apply(lambda row: 1.0*(row['nObservedMuts']- row['nExpectedMuts'])/row['nCases'], axis=1)
combinedDf['geneDisplayName'] = combinedDf.apply(lambda row: 
                                                 row['gene'] if row['geneMutType'] == 'strongly_recurrent'
                                                 else row['gene'] if row['geneObsExpectedRatio'] > .25
                                                 else row['gene'] if row['geneObsExpectedRatio'] < -.1
                                                 else None,axis=1)


In [82]:
combinedDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/observedVsExpectedByGeneHypermutators.tsv', index=False, sep='\t')